In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-finalexam-pp3/menu.json
/kaggle/input/2023-ml-finalexam-pp3/train.csv
/kaggle/input/2023-ml-finalexam-pp3/test.csv
/kaggle/input/2023-ml-finalexam-pp3/sample_submit.csv


In [2]:
#랜덤 시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
#파일 불러오기
import json

with open("/kaggle/input/2023-ml-finalexam-pp3/menu.json") as f:
    total_menu = json.load(f)

In [4]:
#데이터 불러오기
train=pd.read_csv('/kaggle/input/2023-ml-finalexam-pp3/train.csv')
test = pd.read_csv("/kaggle/input/2023-ml-finalexam-pp3/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-finalexam-pp3/sample_submit.csv")

In [5]:
#bow 만들어주기
train_BoW_feature = list()

for idx in range(len(train)):
    menu_list = list()
    BoW = np.zeros((len(total_menu)))
    for col in ['조식메뉴','중식메뉴','석식메뉴']:
        menu_list.extend(train[col].iloc[idx].split(' '))
    menu_list = set(menu_list)
    menu_list.remove('')
    for menu in menu_list:
        idx_ = total_menu.index(menu) # 텀프로젝트 내용을 활용해서 필요한 코드를 작성하세요.
        BoW[idx_] = 1# 텀프로젝트 내용을 활용해서 필요한 코드를 작성하세요.
    train_BoW_feature.append(BoW)

train_BoW_feature = np.array(train_BoW_feature)

test_BoW_feature = list()

for idx in range(len(test)):
    menu_list = list()
    BoW = np.zeros((len(total_menu)))
    for col in ['조식메뉴','중식메뉴','석식메뉴']:
        menu_list.extend(test[col].iloc[idx].split(' '))
    menu_list = set(menu_list)
    menu_list.remove('')
    for menu in menu_list:
        idx_ = total_menu.index(menu) # 텀프로젝트 내용을 활용해서 필요한 코드를 작성하세요.
        BoW[idx_] = 1# 텀프로젝트 내용을 활용해서 필요한 코드를 작성하세요.
    test_BoW_feature.append(BoW)

test_BoW_feature = np.array(test_BoW_feature)

In [6]:
#만들어진 bow shape확인하기
train_BoW_feature.shape

(843, 3294)

In [7]:
test_BoW_feature.shape

(362, 3294)

In [8]:
#pca진행하기 위해 합치기
The_menu = np.concatenate((train_BoW_feature, test_BoW_feature),axis=0)

In [9]:
#pca진행
import numpy as np
from sklearn.decomposition import PCA


pca = PCA(n_components=10)
pca.fit(The_menu)
train_BoW_feature = pca.transform(train_BoW_feature)
test_BoW_feature = pca.transform(test_BoW_feature)

In [10]:
#train 형식 확인
train.head()

,Unnamed: 0,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계
0,0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0
1,1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0
2,2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0
3,3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0
4,4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0


In [11]:
#열 정보 확인
train.columns

Index(['Unnamed: 0', '일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계'],
      dtype='object')

In [12]:
#데이터 분리
y = train['중식계']
train = train[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수']]
test = test[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수']]

In [13]:
#데이터 형식 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   요일              843 non-null    object 
 1   본사정원수           843 non-null    int64  
 2   본사휴가자수          843 non-null    int64  
 3   본사출장자수          843 non-null    int64  
 4   본사시간외근무명령서승인건수  843 non-null    int64  
 5   현본사소속재택근무자수     843 non-null    float64
dtypes: float64(1), int64(4), object(1)
memory usage: 39.6+ KB


In [14]:
#데이터 전처리 - 요일 숫자로
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['요일']) #train에만 하면 안될 수도 있지만, 이번 경우에 요일은 무조건 정해져 있으니까 일단 이렇게 함
train['요일'] = le.transform(train['요일'])
test['요일'] = le.transform(test['요일'])

In [15]:
#np형식으로 변환
train = np.array(train)
test = np.array(test)

In [16]:
#합치기
train = np.concatenate((train, train_BoW_feature),axis=1)
test = np.concatenate((test, test_BoW_feature),axis=1)

In [17]:
#submit확인
submit.head()

,Id,prediction
0,843,0
1,844,0
2,845,0
3,846,0
4,847,0


In [18]:
#여러 모델 돌려보기
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor


lrr = LinearRegression()
lgr = LGBMRegressor()
rfr = RandomForestRegressor()
kmr = KNeighborsRegressor()
vr = VotingRegressor(estimators=[("lrr",lrr),('rfc',rfr),('lgr',lgr)])

models = [lrr, lgr, rfr, kmr, vr]
names = ["lrr","lgr", "rfr", "kmr", 'vr']

for model, name in zip(models,names):
    model.fit(train, y)
    pred = model.predict(test)
    submit["prediction"] = pred
    submit.to_csv("submit_%s.csv"%name,index=False)